In [ ]:
!pip install astropy>=3.1
!pip install sunpy
!pip install imgaug==0.2.5
#!pip install stumpy
!pip install scipy --upgrade
!pip install tslearn
#restart runtime after installation, it should work then
import pandas as pd
#import stumpy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as dates
from matplotlib.patches import Rectangle
import datetime as dt
import scipy
import sklearn
import copy
from tabulate import tabulate

plt.style.use('https://raw.githubusercontent.com/TDAmeritrade/stumpy/main/docs/stumpy.mplstyle')

from google.colab import drive
drive.mount('/content/drive')
DATASET_PATH = '/content/drive/MyDrive/Project/AzureDataset_FunctionCalls/'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import time
start = 0*1440
n = start+1440
m=1

csvName = '0da3f74a774f45236782c3adf430cc25571a0ac9ff7d41afd557efbed747a676'

multiSeriesMain = pd.read_csv(DATASET_PATH+csvName+'.csv')
multiSeriesMain = multiSeriesMain.loc[:, ~multiSeriesMain.columns.str.contains('^Unnamed')].astype(float).reset_index(drop=True).T
multiSeries = multiSeriesMain[start:n]

In [ ]:
#Computes correlation between the two multidimensional time series
#Compares each relevant dimension then returns average correlation value

In [ ]:
#Print Graph of Data
print(len(multiSeriesMain.T))

fig, axs = plt.subplots(len(multiSeriesMain.T), sharex=True, gridspec_kw={'hspace': 0})
if len(multiSeriesMain.T) > 1:
  for k, dim_name in enumerate(multiSeriesMain.columns):
    axs[k].plot(multiSeriesMain[dim_name])
else:
  axs.plot(multiSeriesMain)
plt.show()

20160


KeyboardInterrupt: ignored

Error in callback <function _draw_all_if_interactive at 0x7fd2a8182b80> (for post_execute):


KeyboardInterrupt: ignored

In [ ]:
#Checks number of indices included
def calcNumberIndices(timeSeries, motifs):
  flatMotifs = []
  coveredIndices = []
  for l in motifs:
    for t in l:
      flatMotifs.append(t)
  for f in flatMotifs:
    for i in range(m):
      coveredIndices.append(f+i)
  return len(list(set(coveredIndices)))/len(timeSeries)

In [ ]:
#Finds nearest neighbor in generated clusters
#Returns smallest distance between each point in provided timeseries and individual point within provided clusters. Also returns ratio of neighbors within a distance threshold
def NearestNeighbor(timeSeries, clusters, distanceMax):
  coverage = []
  matches = 0
  for i in range(len(timeSeries)-m):
    point1 = timeSeries[i:i+m].to_numpy().T
    currDist = ["Empty", False, False]
    clusterCount = 0
    for c in clusters:
      innerClusterCount = 0
      for point2 in c:
        point2 = point2.to_numpy().T
        tempDist = np.linalg.norm(point1 - point2)
        if currDist[0] == "Empty":
          currDist = [tempDist, clusterCount, innerClusterCount]
        elif tempDist < currDist[0]:
          currDist = [tempDist, clusterCount, innerClusterCount]
        innerClusterCount = innerClusterCount+1
      clusterCount = clusterCount+1
    #distance threshold
    if currDist[0] <= distanceMax:
      matches = matches + 1
    coverage.append(currDist)
  return coverage, matches/(len(timeSeries)-m)

In [ ]:
#Finds nearest neighbor in generated clusters shapelets
#Returns smallest distance between each point in provided timeseries and individual point within provided clusters. Also returns ratio of neighbors within a distance threshold
def NearestNeighborShapelets(timeSeries, clusters, distanceMax):
  coverage = []
  matches = 0
  for i in range(len(timeSeries)-m):
    point1 = timeSeries[i:i+m].to_numpy().T
    currDist = ["Empty", False, False]
    clusterCount = 0
    for point2 in clusters:
      point2 = point2.to_numpy().T
      tempDist = np.linalg.norm(point1 - point2)
      if currDist[0] == "Empty":
        currDist = [tempDist, clusterCount]
      elif tempDist < currDist[0]:
        currDist = [tempDist, clusterCount]
      clusterCount = clusterCount+1
    #distance threshold
    if currDist[0] <= distanceMax:
      matches = matches + 1
    coverage.append(currDist)
  return coverage, matches/(len(timeSeries)-m)

In [ ]:
#Finds nearest neighbor in generated clusters shapelets
#Returns smallest distance between each point in provided timeseries and individual point within provided clusters. Also returns ratio of neighbors within a distance threshold
def NearestNeighborShapeletsClusters(timeSeries, clusters, clusterNum, clusterDistanceMax):
  coverage = []
  matches = 0
  for i in range(len(timeSeries)-m):
    point1 = timeSeries[i:i+m].to_numpy().T
    currDist = ["Empty", False, False]
    clusterCount = 0
    for point2 in clusters:
      point2 = point2.to_numpy().T
      tempDist = np.linalg.norm(point1 - point2)
      if currDist[0] == "Empty":
        currDist = [tempDist, clusterCount, clusterNum[i]]
      elif tempDist < currDist[0]:
        currDist = [tempDist, clusterCount, clusterNum[i]]
      clusterCount = clusterCount+1
    #distance threshold
    if currDist[0] <= clusterDistanceMax[currDist[2]]:
      matches = matches + 1
    coverage.append(currDist)
  return coverage, matches/(len(timeSeries)-m)

In [ ]:
#Finds Centroid Distances
def GenCentroidDistance(centroids):
  distances = []
  for centroid in centroids:
    tempDistances = []
    for otherCentroid in centroids:
      tempDistances.append(np.linalg.norm(centroid - otherCentroid))
    distances.append(tempDistances)
  return distances

In [ ]:
#Shapelet Generator
#Creates mean shapelet from group
def ShapeletGen(timeSeries, groupIndices):
  shapelet = []
  dimensions = len(timeSeries.T)
  for d in range(dimensions):
    temp = []
    for i in range(m):
      temp.append(0)
    shapelet.append(temp)

  for g in groupIndices:
    for d in range(dimensions):
      for i in range(m):
        shapelet[d][i] = shapelet[d][i] + timeSeries.iloc[g+i,d]

  for d in range(dimensions):
    for i in range(m):
      shapelet[d][i] = shapelet[d][i]/len(groupIndices)
  return shapelet

In [ ]:
#Inner cluster Distance
def ClusterDistance(timeSeries, groupIndices, centroid):
  distances = []
  for g in groupIndices:
    index = timeSeries[g:g+m].to_numpy()
    distances.append(np.linalg.norm(centroid - index))
  return sum(distances)/len(distances)

In [ ]:
def ClusterDistanceMax(timeSeries, groupIndices, centroid):
  if(len(groupIndices) < 1):
    return -1
  else:
    distances = []
    for g in groupIndices:
      index = timeSeries[g:g+m].to_numpy()
      distances.append(np.linalg.norm(centroid - index))
    return max(distances)

In [ ]:
#Convert time series to windows for clustering
def TimeSeriesWindows(timeSeries):
  windows = []
  for t in range(len(timeSeries)-m+1):
    windows.append(timeSeries[t:t+m])
  return windows

In [ ]:
#Correlation Calculations
def ClusterCorrelation(clusters):
  scores = []
  for cluster in clusters:
    for c in range(len(cluster)-1):
      sumScore = 0
      scoreCount = 0
      window1 = cluster[c]
      for win2Index in range(c+1, len(cluster)):
        window2 = cluster[win2Index]
        for dimension in range(len(window1.T)):
          sumScore = sumScore + window1.iloc[:,dimension].corr(window2.iloc[:,dimension])
          scoreCount = scoreCount + 1
    scores.append(sumScore/scoreCount)
  return scores

In [ ]:
#Cluster Correlation from Shapelet
def ShapeletClusterCorrelation(clusters, shapelets):
  scores = []
  test = True
  for count in range(len(shapelets)):
    sumScore = 0
    scoreCount = 0
    window1 = shapelets[count]
    window1 = window1.round().reset_index(drop=True)
    for win2Index in range(len(clusters[count])):
      window2 = clusters[count][win2Index]
      window2 = window2.reset_index(drop=True)
      for dimension in range(len(window1.T)):
        if(not(window1.iloc[:,dimension].corr(window2.iloc[:,dimension]) > 0 or window1.iloc[:,dimension].corr(window2.iloc[:,dimension]) <= 0) and test == True):
          print(window1.iloc[:,dimension])
          print(window2.iloc[:,dimension])
          print(window1.iloc[:,dimension].corr(window2.iloc[:,dimension]))
          test = False
        sumScore = sumScore + window1.iloc[:,dimension].corr(window2.iloc[:,dimension])
        scoreCount = scoreCount + 1
    scores.append(sumScore/scoreCount)
    print(sumScore)
  return scores

In [ ]:
#Euclidean Distance brute forcing
def GenDistancesBruteEuclidean(timeSeries):
  distances = []
  for s in range(len(timeSeries)-(m-1)):
    point1 = timeSeries.iloc[s:s+m, :].to_numpy().T
    temp = []
    for i in range(len(timeSeries)-(m-1)):
      point2 = timeSeries.iloc[i:i+m, :].to_numpy().T
      euclideanDist = np.linalg.norm(point1 - point2)
      temp.append(euclideanDist)
    distances.append(temp)
  return np.array(distances)

In [ ]:
#Binning prerequisites

import time
start = 0*1440
n = start+1440

multiSeriesMainV = pd.read_csv(DATASET_PATH+csvName+'.csv')
multiSeriesMainV = multiSeriesMainV.loc[:, ~multiSeriesMainV.columns.str.contains('^Unnamed')].astype(float).reset_index(drop=True).T
multiSeriesV = multiSeriesMainV.iloc[0,start:n]

In [ ]:
#Binning
def custom_bin_counter(x,y,k=10):
    q75, q25 = np.percentile(x, [75 ,25], axis=0)
    iqr = q75 - q25
    h = (2 * iqr) / ((x.shape[0])**(1/3))
    for i in range(len(h)):
        if h[i] == 0:
            h[i] = 1

    t = {}
    mins = x.min(axis=0)
    maxs = x.max(axis=0)

    ss = []
    total_bins = 1
    for i in range(len(h)):
        total_bins *= (maxs[i]-mins[i])/h[i]
        ss.append((maxs[i]-mins[i], h[i]))

    def get_bin_name(point):
        point = point - mins
        bin = [int(p/h[i]) for i,p in enumerate(point)]
        return bin

    binList = []

    for p in x:
      b = tuple(get_bin_name(p))
      binList.append(b)
      if b not in t:
          t[b] = 0
      t[b] += 1

    binIndexDict = {}
    for key in t:
      binIndexDict[key] = [i for i, x in enumerate(binList) if x==key]
    binWindowDict = {}
    for key in t:
      temp = []
      for index in binIndexDict[key]:
        temp.append(x[index])
      binWindowDict[key] = temp




    k_keys_sorted = sorted(t, key=t.get, reverse=True)[:k]

    x_counter = 0
    for k in k_keys_sorted:
        x_counter += t[k]

    y_counter = 0
    for j in y:
        b = tuple(get_bin_name(j))
        if b in k_keys_sorted:
            y_counter += 1
    return (x_counter/x.shape[0], y_counter/y.shape[0], x.shape[1],total_bins, binIndexDict, binWindowDict, k_keys_sorted,)
sumCoverage = 0
for i in range(0,14):
  multiSeriesOtherV = multiSeriesMainV.iloc[:,i*1440:1440+(i*1440)]
  train_data = multiSeriesV.sort_index().values.transpose()
  test_data = multiSeriesOtherV.sort_index().values.transpose()
  train_coverage, test_coverage, dim, total_bins, binIndexDict, binWindowDict, binKeys = custom_bin_counter(train_data, test_data)
  sumCoverage += test_coverage
  print(train_coverage, test_coverage, dim, total_bins)
print('Sum of Coverage: ',sumCoverage/14)

In [ ]:
# for key in binKeys:
#   print(key)
#   for binIndex in binIndexDict[key]:
#     print(binIndex)

In [ ]:
from tslearn.clustering import TimeSeriesKMeans

In [ ]:
# from scipy.spatial.distance import cdist
# distortions = []
# inertias = []
# mapping1 = {}
# mapping2 = {}
# K = range(1, 30)

# for k in K:
#   # Building and fitting the model
#   model_mem = TimeSeriesKMeans(n_clusters=k, metric="euclidean", n_init=5)

#   newMultiSeries = TimeSeriesWindows(multiSeries)

#   model_mem.fit(newMultiSeries)

#   inertias.append(model_mem.inertia_)

#   #mapping1[k] = sum(np.min(cdist(newMultiSeries, model_mem.cluster_centers_, 'euclidean'), axis=1)) / newMultiSeries.shape[0]
#   mapping2[k] = model_mem.inertia_

In [ ]:
# for keymap, val in mapping2.items():
#     print(f'{keymap} : {val}')

In [ ]:
# plt.plot(K, inertias, 'bx-')
# plt.xlabel('Values of K')
# plt.ylabel('Inertia')
# plt.title('The Elbow Method using Inertia')
# plt.show()
# plt.plot(K[3:30], inertias[3:30], 'bx-')
# plt.xlabel('Values of K')
# plt.ylabel('Inertia')
# plt.title('The Elbow Method using Inertia')
# plt.show()

In [ ]:
#Scale dataset
print(multiSeriesMain.iloc[1441])
def ScaleDataset(scalerVal, firstVal = 1441, lastVal = len(multiSeriesMain)):
  for i in range(firstVal, lastVal):
    multiSeriesMain.iloc[i] = multiSeriesMain.iloc[i] * scalerVal
#ScaleDataset(1.5)
print(multiSeriesMain.iloc[1441])

In [ ]:
#Run Clustering

numClusters = 10

model_mem = TimeSeriesKMeans(n_clusters=numClusters, metric="euclidean", n_init=100)

newMultiSeries = TimeSeriesWindows(multiSeries)

model_mem.fit(newMultiSeries)

labels_mem = pd.DataFrame(model_mem.labels_)
inertia = model_mem.inertia_
centroids = model_mem.cluster_centers_
centroidDistances = GenCentroidDistance(centroids)

clusters = []
for i in range(numClusters):
  tempCluster = []
  for l in range(len(labels_mem)):
    if labels_mem[0][l] == i:
      tempCluster.append(multiSeries[l:l+m])
  clusters.append(tempCluster)

centroidSum = 0
centroidCount = 0
for i in range(len(clusters)-1):
  for l in range(i+1, len(clusters)):
    centroidSum = centroidSum + centroidDistances[i][l]
    centroidCount = centroidCount + 1
centroidAvg = centroidSum/centroidCount

#Centroids generated by TSLearn are mean averages equivalent to shapelets
#shapelets = []
innerDistances = []
for i in range(numClusters):
  clusterIndices = []
  index = 0
  for n in labels_mem[0]:
    if i == n:
      clusterIndices.append(index)
    index = index+1
  innerDistances.append(ClusterDistanceMax(multiSeries, clusterIndices, centroids[i]))
  #shapelet = ShapeletGen(multiSeries, clusterIndices)
  #shapelets.append(shapelet)
#print(tabulate(shapelets))
innerDistancesAvg = []
for i in range(numClusters):
  clusterIndices = []
  index = 0
  for n in labels_mem[0]:
    if i == n:
      clusterIndices.append(index)
    index = index+1
  innerDistancesAvg.append(ClusterDistance(multiSeries, clusterIndices, centroids[i]))

innerDistanceAverage = sum(innerDistances)/len(innerDistances)

#Dataset Coverage
# sumCoverage = 0
# for day in range(14):
#   multiSeriesOther = multiSeriesMain[day*1440:(day+1)*1440]
#   #print("Centroids:\n",tabulate(innerDistances))
#   nearestNeighbors, coverage = NearestNeighbor(multiSeriesOther, clusters, 0.5*m*len(clusters[0][0].T))
#   #print("Nearest Neighbors:\n", tabulate(nearestNeighbors))
#   sumCoverage = coverage + sumCoverage
#   print("Dataset Coverage:\n", day, coverage)
# print("Total Dataset Coverage:\n", sumCoverage/14)

#ShapeletCoverage
clusterShapelets = []
for c in clusters:
  temp = 0
  count = 0
  for win in c:
    if count == 0:
      temp = win.reset_index(drop=True)
    else:
      temp = temp+win.reset_index(drop=True)
    count = count+1
  if count == 0:
    fakeShapelet = []
    for i in range(m):
      fakeShapelet.append(-1)
    clusterShapelets.append(pd.DataFrame(fakeShapelet))
  else:
    temp = temp/count
    clusterShapelets.append(temp)



sumCoverage = 0
sumCoverageAvg = 0
for day in range(14):
  multiSeriesOther = multiSeriesMain[day*1440:(day+1)*1440]
  #print("Centroids:\n",tabulate(innerDistances))
  nearestNeighbors, coverage = NearestNeighborShapeletsClusters(multiSeriesOther, clusterShapelets, labels_mem[0].tolist(), innerDistances)
  nearestNeighborsAvg, coverageAvg = NearestNeighborShapeletsClusters(multiSeriesOther, clusterShapelets, labels_mem[0].tolist(), innerDistancesAvg)
  #print("Nearest Neighbors:\n", tabulate(nearestNeighbors))
  sumCoverage = coverage + sumCoverage
  sumCoverageAvg = coverageAvg + sumCoverageAvg
  #print("Shapelet CoverageAvg:\n", day, coverageAvg)
  print("Shapelet Coverage:\n", day, coverage)
#print("Total Shapelet CoverageAvg:\n", sumCoverageAvg/14)
print("Total Shapelet Coverage:\n", sumCoverage/14)

print("Inertia (Sum of distances of samples to their closest cluster center):", inertia)
print("Centroid Distances:\n", tabulate(centroidDistances))
print("CentroidAvg: ", centroidAvg)
# clusterCorrelation = ShapeletClusterCorrelation(clusters, clusterShapelets)
# print("Avg Cluster Correaltion: ", sum(clusterCorrelation)/len(clusterCorrelation))
print("Inner Distances:\n", tabulate([innerDistances]))
print("Avg Max Inner Distances: ", innerDistanceAverage)
for c in clusters:
  print(len(c))

In [ ]:
print(clusters[0][5])

In [ ]:
clustKeys = []
for e in range(0, numClusters):
  clustKeys.append(e)
clustIndexDict = {}
labels = labels_mem.values.tolist()
for key in clustKeys:
  clustIndexDict[key] = [index for index, x in enumerate(labels) if x[0]==key]
print(clustIndexDict)

In [ ]:
print(labels_mem[0].values.tolist())

In [ ]:
def print_group(listThing, group = 0):
    fig, axs = plt.subplots(len(multiSeries), sharex=True, gridspec_kw={'hspace': 0})

    for k, dim_name in enumerate(multiSeries.columns):
        axs[k].plot(multiSeries[dim_name])
        axs[k].set_xlabel(group, fontsize ='20')

        axs[k + len(multiSeries.T)].plot(multiSeries[k], c='orange')
        axs[k + len(multiSeries.T)].set_xlabel(group, fontsize ='20')

        # for f in listThing:
        #   if(f == group):
        #     axs[k].axvline(x=f[0], linestyle="dashed", c='black')
        #     axs[k + len(multiSeries.T)].axvline(x=f[0], linestyle="dashed", c='black')
        #     axs[k].plot(range(f[0], f[0] + m), multiSeries[dim_name].iloc[f[0] : f[0] + m], c='red', linewidth=4)
        #     axs[k + len(multiSeries.T)].plot(f[0], multiSeries[k, f[0]] + 1, marker="v", markersize=10, color='red')

    plt.show()

In [ ]:
# for key in clustKeys:
#   print_group(labels_mem[0].values.tolist(), key)

In [ ]:
#Cluster Bin Comparison
binList = []
for binkey in binKeys:
  binList.append(binIndexDict[binkey])

clustList = []
for clustkey in clustKeys:
  clustList.append(clustIndexDict[clustkey])

print(binList)
print(clustList)

ratioList = []
countList = []
for clust in clustList:
  ratioTemp = []
  countTemp = []
  for bin in binList:
    ratio = 0
    count = 0
    for c in clust:
      count += bin.count(c)
    if len(clust) == 0:
      ratio = count
    else:
      ratio = count/len(clust)
    countTemp.append(count)
    ratioTemp.append(ratio)
  ratioTemp = np.array(ratioTemp)
  ratioList.append(ratioTemp)
  countTemp = np.array(countTemp)
  countList.append(countTemp)
print(tabulate(pd.DataFrame(ratioList)))
print(tabulate(pd.DataFrame(countList)))
#x is bin, y is cluster, ratio is matches/cluster length

In [ ]:
# #Compute Brute Force Distances
# bruteDistances = GenDistancesBruteEuclidean(multiSeries)

In [ ]:
# #Run Brute Force Neighbor Chain Grouping
# def NeighborChainGrouping(distances, combinations=1):
#   distances_copy = pd.DataFrame(copy.deepcopy(distances))
#   #remove self matches
#   for row in range(len(distances_copy)):
#     for col in range(len(distances_copy)):
#       if row == col:
#         distances_copy[row][col] = 9999999999999
#   nearestNeighbors = []
#   #initial nearest neighbor calculations
#   for row in range(len(distances_copy)):
#     nearestNeighbors.append([row, distances_copy.iloc[:, row].idxmin()])
#   groupings = []
#   for n in range(len(nearestNeighbors)):
#     flag1 = False
#     flag2 = False
#     if len(groupings) == 0:
#       if n == 0:
#         groupings.append([nearestNeighbors[n][0], nearestNeighbors[n][1]])
#     else:
#       if any(n in sublist for sublist in groupings):
#         flag1 = True
#       if any(nearestNeighbors[n][1] in sublist for sublist in groupings):
#         flag2 = True
#       if flag1 and flag2:
#         index1 = None
#         index2 = None
#         for g in range(len(groupings)):
#           if n in groupings[g]:
#             index1 = g
#           if nearestNeighbors[n][1] in groupings[g]:
#             index2 = g
#         if index1 != index2:
#           newList = groupings[index1] + groupings[index2]
#           if index1 > index2:
#             groupings.pop(index1)
#             groupings.pop(index2)
#           else:
#             groupings.pop(index2)
#             groupings.pop(index1)
#           groupings.append(newList)
#       elif flag1:
#         index = None
#         for g in range(len(groupings)):
#           if n in groupings[g]:
#             index = g
#         groupings[index].append(nearestNeighbors[n][1])
#       elif flag2:
#         index = None
#         for g in range(len(groupings)):
#           if nearestNeighbors[n][1] in groupings[g]:
#             index = g
#         groupings[index].append(nearestNeighbors[n][0])
#       else:
#         groupings.append([nearestNeighbors[n][0], nearestNeighbors[n][1]])
#   return groupings
# groupings = NeighborChainGrouping(bruteDistances)
# print(groupings)

In [ ]:
# #Run Brute Force Neighbor Chain Grouping tests
# clusters = []
# for i in range(len(groupings)):
#   tempCluster = []
#   for l in range(len(groupings[i])):
#     tempCluster.append(multiSeries[l:l+m])
#   clusters.append(tempCluster)

# #ShapeletCoverage
# clusterShapelets = []
# for c in clusters:
#   temp = 0
#   count = 0
#   for win in c:
#     if count == 0:
#       temp = win.reset_index(drop=True)
#     else:
#       temp = temp+win.reset_index(drop=True)
#     count = count+1
#   temp = temp/count
#   clusterShapelets.append(temp)




# innerDistances = []
# for i in range(len(groupings)):
#   innerDistances.append(ClusterDistance(multiSeries, groupings[i], clusterShapelets[i]))

# innerDistanceAverage = sum(innerDistances)/len(innerDistances)


# sumCoverage = 0
# for day in range(14):
#   multiSeriesOther = multiSeriesMain[day*1440:(day+1)*1440]
#   #print("Centroids:\n",tabulate(innerDistances))
#   nearestNeighbors, coverage = NearestNeighborShapelets(multiSeriesOther, clusterShapelets, innerDistanceAverage)
#   #print("Nearest Neighbors:\n", tabulate(nearestNeighbors))
#   sumCoverage = coverage + sumCoverage
#   print("Shapelet Coverage:\n", day, coverage)
# print("Total Shapelet Coverage:\n", sumCoverage/14)

# print("Inertia (Sum of distances of samples to their closest cluster center):", inertia)


# centroidDistances = GenCentroidDistance(clusterShapelets)
# centroidSum = 0
# centroidCount = 0
# for i in range(len(clusters)-1):
#   for l in range(i+1, len(clusters)):
#     centroidSum = centroidSum + centroidDistances[i][l]
#     centroidCount = centroidCount + 1
# centroidAvg = centroidSum/centroidCount

# #print("Centroid Distances:\n", tabulate(centroidDistances))
# # clusterCorrelation = ClusterCorrelation(clusters)
# print("CentroidAvg: ", centroidAvg)
# # print("Avg Cluster Correaltion: ", sum(clusterCorrelation)/len(clusterCorrelation))
# print("Inner Distances:\n", tabulate([innerDistances]))
# print("Avg Inner Distances: ", innerDistanceAverage)
# print("Num of Groups: ", len(groupings))

In [ ]:
# print(clusters[0][0].iloc[:,2].corr(clusters[1][1].iloc[:,2]))
# print(type(clusters[0][0].iloc[:,2]))
# print(type(clusters[1][1].iloc[:,2]))